In [14]:
def fitnessFunction(chromosome,network):
    mat = network['mat']
    fitness = 0.0
    for i in range(len(chromosome) - 1):
        fitness += mat[chromosome[i]][chromosome[i+1]]
    return fitness + mat[chromosome[len(chromosome) - 1]][chromosome[0]]


In [18]:
from random import randint, seed


def generateARandomPermutation(n):
    perm = [i for i in range(n)]
    pos1 = randint(0, n - 1)
    pos2 = randint(0, n - 1)
    perm[pos1], perm[pos2] = perm[pos2], perm[pos1]
    return perm

# permutation-based representation
class Chromosome:
    def __init__(self, problParam = None):
        self.__problParam = problParam  #problParam has to store the number of nodes/cities
        self.__repres = generateARandomPermutation(self.__problParam['noNodes'])
        self.__fitness = 0.0
    
    @property
    def repres(self):
        return self.__repres 
    
    @property
    def fitness(self):
        return self.__fitness 
    
    @repres.setter
    def repres(self, l = []):
        self.__repres = l 
    
    @fitness.setter 
    def fitness(self, fit = 0.0):
        self.__fitness = fit 
    
    def crossover(self, c):
        # order XO
        pos1 = randint(-1, self.__problParam['noNodes'] - 1)
        pos2 = randint(-1, self.__problParam['noNodes'] - 1)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1 
        k = 0
        newrepres = self.__repres[pos1 : pos2]
        for el in c.__repres[pos2:] +c.__repres[:pos2]:
            if (el not in newrepres):
                if (len(newrepres) < self.__problParam['noNodes'] - pos1):
                    newrepres.append(el)
                else:
                    newrepres.insert(k, el)
                    k += 1

        offspring = Chromosome(self.__problParam)
        offspring.repres = newrepres
        return offspring
    
    def mutation(self):
        # insert mutation
        pos1 = randint(0, self.__problParam['noNodes'] - 1)
        pos2 = randint(0, self.__problParam['noNodes'] - 1)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1
        el = self.__repres[pos2]
        del self.__repres[pos2]
        self.__repres.insert(pos1 + 1, el)
        
    def __str__(self):
        return "\nChromo: " + str(self.__repres) + " has fit: " + str(self.__fitness)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, c):
        return self.__repres == c.__repres and self.__fitness == c.__fitness

In [19]:
def readNet(fileName):
    f = open(fileName, "r")
    net = {}
    n = int(f.readline())
    net['noNodes'] = n
    mat = []
    for i in range(n):
        mat.append([])
        line = f.readline()
        elems = line.split(",")
        for j in range(n):
            mat[-1].append(int(elems[j]))
    net["mat"] = mat 
    degrees = []
    noEdges = 0
    for i in range(n):
        d = 0
        for j in range(n):
            if (mat[i][j] == 1):
                d += 1
            if (j > i):
                noEdges += mat[i][j]
        degrees.append(d)
    net["noEdges"] = noEdges
    net["degrees"] = degrees
    f.close()
    return net

In [20]:
def read_berlin(fileName):
    coord = {}
    file = open(fileName, "r")
    text = file.read()
    linii = text.split("\n")
    lungime = int(linii[3].split(" ")[1])
    for i in range(6, lungime + 6):
        linie = linii[i].split(" ")
        coord[linie[0]] = (float(linie[1]), float(linie[2]))

    # vrem sa obtinem o matrice de adiacenta
    matrice = [[0 for i in range(lungime)] for j in range(lungime)]
    for i in range(lungime - 1):
        for j in range(i + 1, lungime):
            (x1, y1) = coord[str(i + 1)]
            (x2, y2) = coord[str(j + 1)]
            dist =sqrt((x1 - y1)*(x1 - y1) + (x2 - y2)*(x2 - y2))
            matrice[i][j] = dist
            matrice[j][i] = dist

    net = {}
    net['noNodes'] = lungime
    net['mat'] = matrice
    return net

In [40]:
from random import *

class GA:
    def __init__(self, param = None, problParam = None):
        self.__param = param
        self.__problParam = problParam
        self.__population = []
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self):
        for _ in range(0, self.__param['popSize']):
            c = Chromosome(self.__problParam['network'])
            self.__population.append(c)
    
    def evaluation(self):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres,self.__problParam['network'])
            
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best
        
    def worstChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best

    def selection(self):
        sumfitness =0 
        for c in self.__population:
            sumfitness += c.fitness
        rel_fitness=[]
        for c in self.__population:
            rel_fitness.append(c.fitness/sumfitness) 
        probabilities = [sum(rel_fitness[:i + 1]) for i in range(len(rel_fitness))]

        r = random()
        for i in range(self.__param['popSize']):
            if r <= probabilities[i]:
                return i
        
    def selection1(self):
        pos1 = randint(0, self.__param['popSize'] - 1)
        pos2 = randint(0, self.__param['popSize'] - 1)
        if (self.__population[pos1].fitness < self.__population[pos2].fitness):
            return pos1
        else:
            return pos2 

    
    def oneGeneration(self):
        newPop = []
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()

    def oneGenerationElitism(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection1()]
            p2 = self.__population[self.selection1()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        
    def oneGenerationSteadyState(self):
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            off.fitness = self.__problParam['function'](off.repres,self.__problParam['network'])
            worst = self.worstChromosome()
            if (off.fitness > worst.fitness):
                worst = off

In [41]:
from random import seed 

def findClusters(net,popSize,noGen):
    seed(1)

    # initialise de GA parameters
    gaParam = {'popSize' : popSize, 'noGen' : noGen}
    # problem parameters
    problParam ={'network': net,'function':fitnessFunction}
    ga = GA(gaParam, problParam)
    ga.initialisation()
    ga.evaluation()
    
    for g in range(gaParam['noGen']):
        #logic alg
        ga.oneGenerationElitism()
        bestChromo = ga.bestChromosome()

        print('Best solution in generation ' + str(g) + ' is: x = ' + str(bestChromo.repres) + ' f(x) = ' + str(bestChromo.fitness))

In [52]:
import os
import warnings
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt 
from math import sqrt

crtDir = os.getcwd()
filePathGML = os.path.join(crtDir,'net.in')
network = read_berlin(filePathGML)
bst = findClusters(network,100,300)

neration 224 is: x = [43, 23, 40, 8, 50, 45, 22, 0, 42, 48, 32, 21, 37, 36, 4, 6, 7, 9, 25, 46, 11, 16, 18, 19, 14, 1, 34, 35, 31, 47, 3, 2, 12, 13, 51, 26, 10, 27, 28, 24, 49, 30, 20, 29, 39, 38, 41, 17, 44, 5, 15, 33] f(x) = 28743.60618899398
Best solution in generation 225 is: x = [43, 23, 40, 8, 50, 45, 22, 0, 42, 48, 32, 21, 37, 36, 4, 6, 7, 9, 25, 46, 11, 16, 18, 19, 14, 1, 34, 35, 31, 47, 3, 2, 12, 13, 51, 26, 10, 27, 28, 24, 49, 30, 20, 29, 39, 38, 41, 17, 44, 5, 15, 33] f(x) = 28743.60618899398
Best solution in generation 226 is: x = [43, 23, 40, 8, 50, 45, 22, 0, 42, 48, 32, 21, 37, 36, 4, 6, 7, 9, 25, 46, 11, 16, 18, 19, 14, 1, 34, 35, 31, 47, 3, 2, 12, 13, 51, 26, 10, 27, 28, 24, 49, 30, 20, 29, 39, 38, 41, 17, 44, 5, 15, 33] f(x) = 28743.60618899398
Best solution in generation 227 is: x = [43, 23, 40, 8, 50, 45, 22, 0, 42, 48, 32, 21, 37, 36, 4, 6, 7, 9, 25, 46, 11, 16, 18, 19, 14, 1, 34, 35, 31, 47, 3, 2, 12, 13, 51, 26, 10, 27, 28, 24, 49, 30, 20, 29, 39, 38, 41, 17, 44,